In [1]:
import h5py, json, spacy

import numpy as np
import cPickle as pickle

%matplotlib inline
import matplotlib.pyplot as plt

from model import LSTMModel
from utils import prepare_ques_batch, prepare_im_batch, get_batches_idx

Using TensorFlow backend.


In [2]:
embeddings = spacy.en.English()
word_dim = 300
nb_classes = 1000

In [3]:
h5_img_file_tiny = h5py.File('data/vqa_data_img_vgg_train_tiny.h5', 'r')
fv_im_tiny = h5_img_file_tiny.get('/images_train')

with open('data/qa_data_train_tiny.pkl', 'rb') as fp:
    qa_data_tiny = pickle.load(fp)

json_file = json.load(open('data/vqa_data_prepro.json', 'r'))
ix_to_word = json_file['ix_to_word']
ix_to_ans = json_file['ix_to_ans']

vocab_size = len(ix_to_word)
print "Loading tiny dataset of %d image features and %d question/answer pairs for training." % (len(fv_im_tiny), len(qa_data_tiny)) 

Loading tiny dataset of 100 image features and 300 question/answer pairs for training.


In [22]:
h5_img_file_test_tiny = h5py.File('data/vqa_data_img_vgg_test_tiny.h5', 'r')
fv_im_test_tiny = h5_img_file_test_tiny.get('/images_test')

with open('data/qa_data_test_tiny.pkl', 'rb') as fp:
    qa_data_test_tiny = pickle.load(fp)
    
print "Loading tiny dataset of %d image features and %d question/answer pairs for testing" % (len(fv_im_test_tiny), len(qa_data_test_tiny)) 

Loading tiny dataset of 100 image features and 300 question/answer pairs for testing


In [31]:
model = LSTMModel(vocab_size = vocab_size)
model.build()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
bidirectional_3 (Bidirectional)  (None, 1024)          3330048                                      
____________________________________________________________________________________________________
maxpooling2d_3 (MaxPooling2D)    (None, 7, 7, 512)     0                                            
____________________________________________________________________________________________________
flatten_3 (Flatten)              (None, 25088)         0                                            
____________________________________________________________________________________________________
dense_13 (Dense)                 (None, 4096)          102764544                                    
___________________________________________________________________________________________

In [32]:
max_batch_len = 26
batch_size = 50
epochs = 50

In [33]:
questions, ques_len, im_ix, ans = zip(*qa_data_tiny)

for idx in xrange(epochs):
    batch = get_batches_idx(300,batch_size, True)
    for _, train_index in batch:
        uidx += 1
        n_samples += len(batch)
        ques_batch = [questions[t] for t in train_index]
        ques_len_batch = [ques_len[t] for t in train_index]
        img_batch = [im_ix[t] for t in train_index]
        ans_batch = [ans[t] for t in train_index]
        X_ques = prepare_ques_batch(ques_batch, ques_len_batch, max_batch_len, embeddings, word_dim, ix_to_word)
        X_im = prepare_im_batch(fv_im_tiny, img_batch)
        y = np.zeros((len(ans_batch), nb_classes))
        y[np.arange(len(ans_batch)), ans_batch] = 1
        loss = model.train_on_batch(X_ques, X_im, y)
    print 'Epoch ', (idx+1), 'Loss', loss[0], 'Accurancy', loss[1]

Epoch  1 Loss 7.18617 Accurancy 0.06
Epoch  2 Loss 5.31412 Accurancy 0.2
Epoch  3 Loss 3.30081 Accurancy 0.38
Epoch  4 Loss 2.21071 Accurancy 0.54
Epoch  5 Loss 1.43598 Accurancy 0.68
Epoch  6 Loss 1.31962 Accurancy 0.66
Epoch  7 Loss 1.02898 Accurancy 0.72
Epoch  8 Loss 0.912009 Accurancy 0.84
Epoch  9 Loss 0.820408 Accurancy 0.78
Epoch  10 Loss 0.613635 Accurancy 0.82
Epoch  11 Loss 0.565082 Accurancy 0.8
Epoch  12 Loss 0.51537 Accurancy 0.88
Epoch  13 Loss 0.332987 Accurancy 0.96
Epoch  14 Loss 0.369069 Accurancy 0.92
Epoch  15 Loss 0.610501 Accurancy 0.8
Epoch  16 Loss 0.205189 Accurancy 0.9
Epoch  17 Loss 0.353496 Accurancy 0.92
Epoch  18 Loss 0.14289 Accurancy 0.94
Epoch  19 Loss 0.406284 Accurancy 0.92
Epoch  20 Loss 0.165164 Accurancy 0.94
Epoch  21 Loss 0.316766 Accurancy 0.94
Epoch  22 Loss 0.129356 Accurancy 0.94
Epoch  23 Loss 0.285355 Accurancy 0.92
Epoch  24 Loss 0.130115 Accurancy 0.96
Epoch  25 Loss 0.115229 Accurancy 0.98
Epoch  26 Loss 0.218955 Accurancy 0.96
Epoch  2

In [34]:
questions, ques_len, im_ix, ans = zip(*qa_data_test_tiny)

X_ques_test = prepare_ques_batch(questions, ques_len, max_ques_len, embeddings, word_dim, ix_to_word)
X_im_test = prepare_im_batch(fv_im_test_tiny, im_ix)
y_test = np.zeros((len(ans), nb_classes))
y_test[np.arange(len(ans)), [494 if a > 1000 else a for a in ans]] = 1

In [35]:
loss, acc = model.evaluate(X_ques_test, X_im_test, y_test)

300/300 [==============================] - 4s     


In [36]:
print acc

0.25
